# Ensemble M1

Importing the necessary libraries

In [3]:
import numpy as np
import pandas as pd
import pathlib
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch import nn ,cuda
from torch.utils.data import DataLoader,Dataset,RandomSampler, SequentialSampler

from sklearn.metrics import precision_recall_fscore_support, classification_report
import nltk.corpus
from sklearn import metrics
from scipy.speci# Roberta M4 (with class weight)

Importing the necessary librariesal import softmax


from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer,TrainerCallback
import glob
from datasets import Dataset

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from transformers import EvalPrediction   
from sklearn.metrics import confusion_matrix

In [2]:
torch.cuda.empty_cache()

## 1. Loading the data

In [4]:
## 1. Loading the datafolder_path = 'data/clean/'
file_pattern = folder_path + '*.csv'
csv_files = glob.glob(file_pattern)

for csv_file in csv_files:
    if 'train' in csv_file:
        df_train = pd.read_csv(csv_file)
    elif 'val' in csv_file:
        df_val = pd.read_csv(csv_file)
    else:
        df_test = pd.read_csv(csv_file)

#### Storing all 28 labels into variable target_cols

In [6]:
target_cols = [col for col in df_train.columns if col not in ['clean_text']]

## 2. Model Building

In [7]:
#define path to load tokenizer and model
roberta_pth = "./roberta_M2_transformer/"
distilbert_pth = "./distilbert_M2_transformer/"
roberta_m4_path = "./roberta_M4_transformer/"

In [8]:
# Load the Roberta tokenizer
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_pth)

# Load the Roberta model
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_pth)

In [9]:
# Load the tokenizer
distilbert_tokenizer = AutoTokenizer.from_pretrained(distilbert_pth)

# Load the model
distilbert_model = AutoModelForSequenceClassification.from_pretrained(distilbert_pth)

In [10]:
# Load the tokenizer
roberta_tokenizer_2 = AutoTokenizer.from_pretrained(roberta_m4_path)

# Load the model
roberta_model_2 = AutoModelForSequenceClassification.from_pretrained(roberta_m4_path)

In [11]:
#function to calculate each emotion label metrics on test set
def calc_label_metrics(label, y_targets, y_preds, threshold):
    return {
        "label": label,
        "accuracy": metrics.accuracy_score(y_targets, y_preds),
        "precision": metrics.precision_score(y_targets, y_preds, zero_division=0),
        "recall": metrics.recall_score(y_targets, y_preds, zero_division=0),
        "f1": metrics.f1_score(y_targets, y_preds, zero_division=0),
        "mcc": metrics.matthews_corrcoef(y_targets, y_preds),
        "support": y_targets.sum(),
        "threshold": threshold,
    }

In [12]:
#function to calculate overall metric on test set
def calc_test_metrics(y_test, target_cols):
    threshold = 0.6
    # compute metrics
    y_true = df_test[target_cols].values
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    precision_micro = precision_score(y_true, y_pred, average='micro')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'recall' : recall_micro,
               'precision': precision_micro,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Value'])

    display(metrics_df)
    
    results = []
    for label_index, label in enumerate(target_cols):
        y_targets, y_preds = y_true[:, label_index], y_pred[:, label_index]
        results.append(calc_label_metrics(label, y_targets, y_preds, threshold))

    per_label_results = pd.DataFrame(results, index=target_cols)
    display(per_label_results.drop(columns=["label"]).round(3))
    
    return y_true, y_pred

In [13]:
#store the model and tokenizer in array
models = [roberta_model, distilbert_model, roberta_model_2]
tokenizers = [roberta_tokenizer, distilbert_tokenizer, roberta_tokenizer_2]

## 3. Ensemble Fuzzy Integral Logic

In [ ]:
#genarate weight for models
def generate_cardinality(N, p = 2):
    return [(x/ N)**p for x in np.arange(N, 0, -1)]

In [14]:
#sort the prediction for each model and keep pred above the threshold value
def fuzzy_integral_generalized(X, measure, axis = 0, f1 = np.minimum, f2 = np.amax, keepdims=True):
    X_sorted = np.sort(X, axis = axis)
    return f2(f1(np.take(X_sorted, np.arange(0, X_sorted.shape[axis]), axis), measure), axis=axis, keepdims=keepdims)

def predict(ensemble_prob):
    predictions = np.zeros_like(ensemble_prob)
    
    for i in range(ensemble_prob.shape[0]):
        temp = ensemble_prob[i]
        # Thresholding: If value is greater than threshold, substitute with 1, else 0
        predictions[i] = (temp >= 0.6).astype(int)
    
    return predictions

In [15]:
#Load each model tokenizer, feed data into the model for prediction
#Apply the fuzzy integral logic
#Make predictions
def my_ensemble(models, tokenizers, target_cols, df, y_true):
    
    count = 0
    for model, tokenizer in zip(models, tokenizers):
        count = count + 1
        inputs = tokenizer(list(df['clean_text']), return_tensors="pt", padding=True, truncation=True)

        with torch.no_grad():
            outputs = model(**inputs)

        sigmoid = torch.nn.Sigmoid()
        probs = sigmoid(torch.Tensor(outputs.logits))
        probs_np = probs.cpu().detach().numpy()
        
        if count == 1: 
            prob1 = probs_np
        elif count == 2 :
            prob2 = probs_np
        else: 
            prob3 = probs_np
        
    num_classes = prob1.shape[1] #Get number of labels
    Y = np.zeros(prob1.shape,dtype=float) #create a mask with 0s
    
    for samples in range(prob1.shape[0]): #loops the rows in the test set
        for classes in range(prob1.shape[1]): #loop the classes
            X = np.array([prob1[samples][classes], prob2[samples][classes],  prob3[samples][classes]])
            measure = [0.5, 1.5, 1.8]
            X_agg = fuzzy_integral_generalized(X,measure)
            Y[samples][classes] = X_agg
    
    y_predictions = predict(Y)    
    
    correct = np.where(y_predictions == y_true)[0].shape[0]
    total = y_true.shape[0]

    accuracy = accuracy_score(y_true, y_predictions)
    print("Accuracy = ",accuracy)

    return y_predictions

In [16]:
y_pred = my_ensemble(models, tokenizers, target_cols, df_test, df_test[target_cols].values)

Accuracy =  0.4790860512253547


In [17]:
targets, predictions = calc_test_metrics(y_pred, target_cols)

,Value
f1,0.600494
recall,0.556802
precision,0.651627
roc_auc,0.771932
accuracy,0.479086


,accuracy,precision,recall,f1,mcc,support,threshold
admiration,0.940,0.653,0.762,0.703,0.673,504,0.6
amusement,0.982,0.766,0.894,0.825,0.818,264,0.6
anger,0.966,0.536,0.485,0.509,0.492,198,0.6
annoyance,0.939,0.467,0.222,0.301,0.294,320,0.6
approval,0.939,0.543,0.342,0.420,0.401,351,0.6
caring,0.975,0.495,0.348,0.409,0.403,135,0.6
confusion,0.972,0.508,0.418,0.459,0.447,153,0.6
curiosity,0.943,0.441,0.352,0.391,0.364,284,0.6
desire,0.987,0.635,0.398,0.489,0.496,83,0.6
disappointment,0.974,0.596,0.185,0.283,0.323,151,0.6


In [18]:
# Create a DataFrame to store actual labels and predicted labels
final_df = pd.DataFrame({
    'Actual': [list(np.where(targets[i])[0]) for i in range(len(targets))],
    'Predicted': [list(np.where(predictions[i])[0]) for i in range(len(predictions))]
})

# Map label indices to label names in the 'Actual' column
final_df['Actual'] = final_df['Actual'].apply(lambda indices: [target_cols[idx] for idx in indices])

# Map label indices to label names in the 'Predicted' column
final_df['Predicted'] = final_df['Predicted'].apply(lambda indices: [target_cols[idx] for idx in indices])

# Display the results DataFrame
print("Results DataFrame:")
print(final_df.head(10))

Results DataFrame:
         Actual                Predicted
0     [sadness]          [love, remorse]
1  [admiration]             [admiration]
2  [excitement]               [optimism]
3   [gratitude]              [gratitude]
4     [neutral]                [neutral]
5   [gratitude]              [gratitude]
6   [gratitude]              [gratitude]
7   [gratitude]  [admiration, gratitude]
8     [remorse]                [remorse]
9     [sadness]                [sadness]


In [19]:
# Concatenate the label DataFrame with the original DataFramev
val_df_terms = df_test['clean_text']
result_df = pd.concat([val_df_terms, final_df], axis=1)

In [20]:
result_df

,clean_text,Actual,Predicted
0,i am really sorry about your situation frown s...,[sadness],"[love, remorse]"
1,it is wonderful because it is awful at not with,[admiration],[admiration]
2,kings fan here good luck to you guys will be a...,[excitement],[optimism]
3,i did not know that thank you for teaching me ...,[gratitude],[gratitude]
4,they got bored from haunting earth for thousan...,[neutral],[neutral]
...,...,...,...
5422,thanks i was diagnosed with bp 1 after the hos...,[gratitude],[gratitude]
5423,well that makes sense,[approval],[approval]
5424,daddy issues name,[neutral],[neutral]
5425,so glad i discovered that subreddit a couple m...,[admiration],"[admiration, joy]"


In [21]:
#save result to csv
result_df.to_csv('output_ensemble_1.csv', index=False, sep=';')